# Task 1


In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_parquet("data/spx_ivs_2023-02.parquet")
df["rate"] = np.log(df["underlyingforwardprice"] / df["underlyingprice"]) /  (30 / 365)
df["test normalized moneyness"] = np.log(df["forwardMoneyness"]) / (np.sqrt(30/365) * df['implVol'])

#df["loctimestamp"] = pd.to_datetime(df["loctimestamp"])
#df["loctimestamp"].rename("date")

grouped_by_day = {date : group for date,group in df.groupby('loctimestamp')}

# strike price
K = df["strike"].copy()


S_t = df["underlyingprice"].copy()

C = df[df["putcall"] == "C"]["implPrice"].copy()

# put price C(t, tau, K) depending of an underlying asset with price S(t), date to maturity tau, strike price K
P = df[df["putcall"] == "P"]["implPrice"].copy()

df

,Symbol,loctimestamp,putcall,daystomaturity,implVol,implPrice,strike,forwardMoneyness,normalizedMoneyness,underlyingprice,underlyingforwardprice,rate,test normalized moneyness
0,^SPX,2023-02-01,P,30,0.546362,0.162479,2572.16,0.620158,-10.00,4138.45,4147.58,0.026812,-3.050241
1,^SPX,2023-02-01,P,30,0.537310,0.176469,2603.06,0.627610,-9.75,4138.45,4147.58,0.026812,-3.024087
2,^SPX,2023-02-01,P,30,0.528999,0.195240,2634.34,0.635152,-9.50,4138.45,4147.58,0.026812,-2.992833
3,^SPX,2023-02-01,P,30,0.520193,0.214198,2666.00,0.642784,-9.25,4138.45,4147.58,0.026812,-2.963405
4,^SPX,2023-02-01,P,30,0.510792,0.232522,2698.03,0.650507,-9.00,4138.45,4147.58,0.026812,-2.936388
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1078,^SPX,2023-02-28,C,30,0.171093,0.032910,4672.92,1.172590,3.00,3974.66,3985.14,0.032038,3.245918
1079,^SPX,2023-02-28,C,30,0.179106,0.022557,4735.33,1.188250,3.25,3974.66,3985.14,0.032038,3.359066
1080,^SPX,2023-02-28,C,30,0.186005,0.014515,4798.57,1.204120,3.50,3974.66,3985.14,0.032038,3.483274
1081,^SPX,2023-02-28,C,30,0.191016,0.008167,4862.66,1.220200,3.75,3974.66,3985.14,0.032038,3.634138


In [21]:
df_firstday = df[df["loctimestamp"] == pd.Timestamp("2023-02-01")].copy()

In [35]:
#unnötig
def trap_integral(y, x): #integrate y over x by using trapezoids as approximations

    df_total = pd.concat([x.copy(), y.copy()], axis=1)
    df_total.columns = ['x', 'y']

    # Ensure x is sorted (is the case in the data already)
    df_total = df_total.sort_values("x")

    df_total["x_shifted"] = df_total["x"].shift()
    df_total["y_shifted"] = df_total["y"].shift()
    df_total = df_total.dropna()

    integral = (
            (df_total["x"] - df_total["x_shifted"]) *
            (df_total["y"] + df_total["y_shifted"]) / 2
    ).sum()
    return integral


def calculate_V(df_one_day): #df_one_day only contains data from one day

    # underlying price at time t
    S_t = df_one_day["underlyingprice"].copy()

    # strike price
    K = df_one_day["strike"].copy()

    # call price C(t, tau, K) depending of an underlying asset with price S(t), date to maturity tau, strike price K
    C = df_one_day[df_one_day["putcall"] == "C"]["implPrice"].copy()

    # put price C(t, tau, K) depending of an underlying asset with price S(t), date to maturity tau, strike price K
    P = df_one_day[df_one_day["putcall"] == "P"]["implPrice"].copy()

    # formula for integrand in integral from S_t to infinity
    puts = (2 * (1 + np.log(S_t / K)) / np.square(K) * P).dropna()

    # formula for integrand in integral from S_t to infinity
    calls = (2 * (1 - np.log(K / S_t)) / np.square(K) * C).dropna()

    x = K.copy()

    # concat works as dataset is sorted in a way that puts come before calls.
    y = pd.concat([puts, calls]).copy()

    #return trap_integral(y,x)
    return np.trapezoid(y,x)

for timestamp, df_day in df.groupby('loctimestamp'):
    print(f"{timestamp.strftime("%Y-%m-%d")} : {calculate_V(df_day):.6f}")



2023-02-01 : 0.002881
2023-02-02 : 0.003308
2023-02-03 : 0.002807
2023-02-06 : 0.003302
2023-02-07 : 0.002921
2023-02-08 : 0.003253
2023-02-09 : 0.003708
2023-02-10 : 0.003425
2023-02-13 : 0.003874
2023-02-14 : 0.002878
2023-02-15 : 0.002741
2023-02-16 : 0.003228
2023-02-17 : 0.002966
2023-02-21 : 0.004187
2023-02-22 : 0.004039
2023-02-23 : 0.003411
2023-02-24 : 0.003524
2023-02-27 : 0.003555
2023-02-28 : 0.003355


In [34]:
def calculate_W(df_one_day):

    S_t = df_one_day["underlyingprice"].copy()
    K = df_one_day["strike"].copy()
    C = df_one_day[df_one_day["putcall"] == "C"]["implPrice"].copy()
    P = df_one_day[df_one_day["putcall"] == "P"]["implPrice"].copy()

    # formula for integrand in integral from S_t to infinity
    puts = ((6 * np.log(S_t / K) + 3 * (np.log(S_t / K)) ** 2 ) / (K ** 2) * P).dropna()

    calls = (( 6 * np.log(K / S_t) - 3 * (np.log(K / S_t)) ** 2) / (K ** 2) * C).dropna()


    puts_integral = np.trapezoid(puts, K[df_one_day["putcall"] == "P"])
    calls_integral = np.trapezoid(calls, K[df_one_day["putcall"] == "C"])


    return calls_integral - puts_integral


for timestamp, df_day in df.groupby('loctimestamp'):
    print(f"{timestamp.strftime("%Y-%m-%d")} : {calculate_W(df_day):.6f}")


2023-02-01 : -0.000271
2023-02-02 : -0.000343
2023-02-03 : -0.000270
2023-02-06 : -0.000330
2023-02-07 : -0.000281
2023-02-08 : -0.000341
2023-02-09 : -0.000414
2023-02-10 : -0.000367
2023-02-13 : -0.000424
2023-02-14 : -0.000282
2023-02-15 : -0.000260
2023-02-16 : -0.000389
2023-02-17 : -0.000317
2023-02-21 : -0.000548
2023-02-22 : -0.000527
2023-02-23 : -0.000364
2023-02-24 : -0.000387
2023-02-27 : -0.000377
2023-02-28 : -0.000328


In [37]:

def calculate_X(df_one_day):
    S_t = df_one_day["underlyingprice"].copy()
    K = df_one_day["strike"].copy()
    C = df_one_day[df_one_day["putcall"] == "C"]["implPrice"].copy()
    P = df_one_day[df_one_day["putcall"] == "P"]["implPrice"].copy()

    # formula for integrand in integral from S_t to infinity
    puts = ((12 * np.log(S_t / K) ** 2 + 4 * np.log(S_t / K) ** 3) / (K ** 2) * P).dropna()

    calls = ((12 * np.log(K / S_t) ** 2 - 4 * np.log(K / S_t) ** 3 ) / (K ** 2) * C).dropna()

    puts_integral = np.trapezoid(puts, K[df_one_day["putcall"] == "P"])
    calls_integral = np.trapezoid(calls, K[df_one_day["putcall"] == "C"])

    return calls_integral + puts_integral


for timestamp, df_day in df.groupby('loctimestamp'):
    print(f"{timestamp.strftime("%Y-%m-%d")} : {calculate_X(df_day):.6f}")

2023-02-01 : 0.000111
2023-02-02 : 0.000150
2023-02-03 : 0.000108
2023-02-06 : 0.000144
2023-02-07 : 0.000119
2023-02-08 : 0.000148
2023-02-09 : 0.000188
2023-02-10 : 0.000164
2023-02-13 : 0.000203
2023-02-14 : 0.000116
2023-02-15 : 0.000105
2023-02-16 : 0.000164
2023-02-17 : 0.000128
2023-02-21 : 0.000268
2023-02-22 : 0.000252
2023-02-23 : 0.000160
2023-02-24 : 0.000182
2023-02-27 : 0.000182
2023-02-28 : 0.000156


In [42]:
df_riskfree_yields = pd.read_csv("data/riskfree_30d_2023-02.csv")

df_riskfree_yields['exp_rt'] = np.exp((df_riskfree_yields['yld_pct_annual'] / 100) * (df_riskfree_yields['daystomaturity'] / 360))

df_riskfree_yields


,date,daystomaturity,yld_pct_annual,exp_rt
0,2023-02-01,30,4.680,1.003908
1,2023-02-02,30,4.714,1.003936
2,2023-02-03,30,4.629,1.003865
3,2023-02-06,30,4.686,1.003913
4,2023-02-07,30,4.685,1.003912
5,2023-02-08,30,4.710,1.003933
6,2023-02-09,30,4.730,1.003949
7,2023-02-10,30,4.700,1.003924
8,2023-02-13,30,4.756,1.003971
9,2023-02-14,30,4.680,1.003908
